# Emerging Technolgoies

### Libraries 

In [1]:
# Random selections.
import random

# Numerical arrays.
import numpy as np

## Problem 1: Generating Random Boolean Functions


### Define the four constant and balanced functions 

In [ ]:
# Constant function 1.
# Constant functions do not depend on any input
# they always return the same value (0 or 1), regardless of
# the values of their arguments.
def f1(a,b,c,d):
  """Takes a single bit but always returns 0."""
  return 0

# Balanced function 1.
def f2(a,b,c,d):
  """Identity function."""
  return a

# Balanced function 2.
def f3(a,b,c,d):
  """Not function."""
  return a ^ b ^ c ^ d

# constant function 2. 
def f4(a,b,c,d):
  """Constant 1 function."""
  return 1

### The random_constant_balanced function returns a randomly selected function 

In [3]:
def random_constant_balanced():
  """Randomly returns one of the four functions defined above."""
  # List of potential functions.
  functions = [f1, f2, f3, f4]
  # Return element at random.
  random_function = random.choice(functions)
  return random_function

  # for each function need to return whether it is constant or balanced by passing in the four difference values of a,b,c,d
  

random_constant_balanced()

<function __main__.f3(a, b, c, d)>

### Test the random_constant_balanced function 

In [ ]:
def test_function_type(f):
    results = []
    # All possible input combinations for four binary variables (a, b, c, d)
    results = [
        f(0,0,0,0), f(0,0,0,1), f(0,0,1,0), f(0,0,1,1),
        f(0,1,0,0), f(0,1,0,1), f(0,1,1,0), f(0,1,1,1),
        f(1,0,0,0), f(1,0,0,1), f(1,0,1,0), f(1,0,1,1),
        f(1,1,0,0), f(1,1,0,1), f(1,1,1,0), f(1,1,1,1)
    ]
    
    count_ones = results.count(1)
    count_zeros = results.count(0)
    if count_ones == 16 or count_zeros == 16:
        print("constant:", results)
    elif count_ones == 8 and count_zeros == 8:
        print("balanced:", results)
    else:
        print("neither (something is wrong):", results)

## Problem 2: Classical Testing for Function Type


## Problem 3: Quantum Oracles


## Problem 4: Deutsch's Algorithm with Qiskit


## Problem 5: Scaling to the Deutsch–Jozsa Algorithm
